In [1]:
"""
!pip install gptpdf 


동작 원리 
1. non-text area를 mark 
2. GPT4o 모델 사용해서 markdown 생성 


def parse_pdf(
        pdf_path: str,
        output_dir: str = './',
        prompt: Optional[Dict] = None,
        api_key: Optional[str] = None,
        base_url: Optional[str] = None,
        model: str = 'gpt-4o',
        verbose: bool = False,
        gpt_worker: int = 1
) -> Tuple[str, List[str]]:
"""

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached PyMuPDF-1.24.10-cp312-none-win_amd64.whl.metadata (3.4 kB)
  Using cached tiktoken-0.7.0-cp312-cp312-win_amd64.whl.metadata (6.8 kB)
  Using cached PyMuPDFb-1.24.10-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached aiohttp-3.10.5-cp312-cp312-win_amd64.whl.metadata (7.8 kB)
  Using cached dataclasses_json-0.6.7-py3-non

2024-09-13 15:43:27,599 - INFO - parse page: 0
2024-09-13 15:45:01,491 - INFO - parse page: 1
2024-09-13 15:47:01,427 - INFO - parse page: 2
2024-09-13 15:48:21,871 - INFO - parse page: 3
2024-09-13 15:49:23,675 - INFO - parse page: 4
2024-09-13 15:49:25,362 - INFO - parse page: 5
2024-09-13 15:49:28,769 - INFO - parse page: 6
2024-09-13 15:49:31,530 - INFO - parse page: 7
2024-09-13 15:49:36,380 - INFO - parse page: 8
2024-09-13 15:49:42,043 - INFO - parse page: 9
2024-09-13 15:49:46,721 - INFO - prompt is not provided, using default prompt.
2024-09-13 15:49:46,721 - INFO - rect_prompt is not provided, using default prompt.
2024-09-13 15:49:46,722 - INFO - role_prompt is not provided, using default prompt.
2024-09-13 15:49:46,723 - INFO - gpt parse page: 0
2024-09-13 15:49:55,681 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-13 15:50:03,159 - INFO - gpt parse page: 1
2024-09-13 15:50:06,742 - INFO - HTTP Request: POST https://api.open

# 목차

## 별지

### 증권신고서

- **별지 제1.1호 서식**: 주권회사의 주의의무 이행서류 ........................................................... 37
- **별지 제1.2호 서식**: 회계감사인의 재무확인서 ............................................................ 41
- **별지 제2호 서식**: 증권신고서(IDS적용 외국기업) .................................................... 44
- **별지 제3호 서식**: 증권신고서(외국기업 외의 외국법인등) ....................................... 77
- **별지 제4호 서식**: 증권신고서(파생결합증권 등) ...................................................... 91
- **별지 제5호 서식**: 증권(일괄)신고서(집합투자증권-신탁형) ..................................... 110
- **별지 제6호 서식**: 증권(일괄)신고서(집합투자증권-회사형) .................................... 125
- **별지 제7호 서식**: 증권신고서(유동화증권) ............................................................... 141
- **별지 제8호 서식**: 삭제<2021.7.13.> .......................................................................... 154
- **별지 제9호 서식**: 증권신고서(합병등) ....................................................................... 158
- **별지 제10호 서식**: 증권신고서(집합투자증권 합병) ..........

In [3]:
with open('gptpdf_output.txt', 'w') as file:
    file.write(content)

In [1]:
import os
import re
from typing import List, Tuple, Optional, Dict
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
import fitz  # PyMuPDF
import shapely.geometry as sg
from shapely.geometry.base import BaseGeometry
from shapely.validation import explain_validity
import concurrent.futures



DEFAULT_PROMPT = """마크다운 문법을 사용하여 이미지에서 인식된 텍스트를 마크다운 형식으로 출력하세요. 다음을 준수해야 합니다:
1. 인식된 이미지의 언어와 동일한 언어로 출력합니다. 예를 들어, 한국어로 인식된 필드는 한국어로 출력해야 합니다.
2. 설명이나 관련 없는 텍스트를 출력하지 말고, 이미지의 내용을 직접 출력합니다. 예를 들어, "다음은 이미지 내용을 기반으로 생성된 마크다운 텍스트입니다:"와 같은 예시는 금지되며, 대신 마크다운을 직접 출력해야 합니다.
3. 내용은 ```markdown ```으로 감싸지 말고, 단락 수식은 $$ $$ 형식으로, 인라인 수식은 $ $ 형식으로 사용하며, 긴 직선과 페이지 번호는 무시합니다.
다시 강조하지만, 설명이나 관련 없는 텍스트를 출력하지 말고, 이미지의 내용을 직접 출력하세요.
"""
DEFAULT_RECT_PROMPT = """이미지에서 빨간색 상자와 이름(%s)으로 표시된 영역이 있습니다. 영역이 표나 이미지인 경우 ![]() 형식으로 출력 내용에 삽입하고, 그렇지 않은 경우 텍스트 내용을 직접 출력하세요.
"""
DEFAULT_ROLE_PROMPT = """당신은 PDF 문서 파서입니다. 마크다운과 LaTeX 문법을 사용하여 입력한 이미지의 내용을 출력하세요.
"""


def _is_near(rect1: BaseGeometry, rect2: BaseGeometry, distance: float = 20) -> bool:
    """
    두 사각형이 서로 가까운지 확인합니다. 두 사각형 사이의 거리가 목표 거리보다 작으면 True를 반환합니다.
    """
    return rect1.buffer(0.1).distance(rect2.buffer(0.1)) < distance


def _is_horizontal_near(rect1: BaseGeometry, rect2: BaseGeometry, distance: float = 100) -> bool:
    """
    두 사각형이 수평으로 가까운지 확인합니다. 하나가 수평선인 경우에만 적용됩니다.
    """
    result = False
    if abs(rect1.bounds[3] - rect1.bounds[1]) < 0.1 or abs(rect2.bounds[3] - rect2.bounds[1]) < 0.1:
        if abs(rect1.bounds[0] - rect2.bounds[0]) < 0.1 and abs(rect1.bounds[2] - rect2.bounds[2]) < 0.1:
            result = abs(rect1.bounds[3] - rect2.bounds[3]) < distance
    return result


def _union_rects(rect1: BaseGeometry, rect2: BaseGeometry) -> BaseGeometry:
    """
    두 사각형을 합칩니다.
    """
    return sg.box(*(rect1.union(rect2).bounds))


def _merge_rects(rect_list: List[BaseGeometry], distance: float = 20, horizontal_distance: Optional[float] = None) -> \
        List[BaseGeometry]:
    """
    리스트의 사각형들을 병합합니다. 두 사각형 사이의 거리가 목표 거리보다 작으면 병합합니다.
    """
    merged = True
    while merged:
        merged = False
        new_rect_list = []
        while rect_list:
            rect = rect_list.pop(0)
            for other_rect in rect_list:
                if _is_near(rect, other_rect, distance) or (
                        horizontal_distance and _is_horizontal_near(rect, other_rect, horizontal_distance)):
                    rect = _union_rects(rect, other_rect)
                    rect_list.remove(other_rect)
                    merged = True
            new_rect_list.append(rect)
        rect_list = new_rect_list
    return rect_list


def _adsorb_rects_to_rects(source_rects: List[BaseGeometry], target_rects: List[BaseGeometry], distance: float = 10) -> \
        Tuple[List[BaseGeometry], List[BaseGeometry]]:
    """
    한 세트의 사각형을 다른 세트의 사각형에 흡착시킵니다.
    """
    new_source_rects = []
    for text_area_rect in source_rects:
        adsorbed = False
        for index, rect in enumerate(target_rects):
            if _is_near(text_area_rect, rect, distance):
                rect = _union_rects(text_area_rect, rect)
                target_rects[index] = rect
                adsorbed = True
                break
        if not adsorbed:
            new_source_rects.append(text_area_rect)
    return new_source_rects, target_rects


def _parse_rects(page: fitz.Page) -> List[Tuple[float, float, float, float]]:
    """
    페이지의 도형을 파싱하고 인접한 사각형을 병합합니다.
    """

    # 도형 내용을 추출합니다.
    drawings = page.get_drawings()

    # 길이가 30보다 작은 수평선을 무시합니다.
    is_short_line = lambda x: abs(x['rect'][3] - x['rect'][1]) < 1 and abs(x['rect'][2] - x['rect'][0]) < 30
    drawings = [drawing for drawing in drawings if not is_short_line(drawing)]

    # shapely의 사각형으로 변환합니다.
    rect_list = [sg.box(*drawing['rect']) for drawing in drawings]

    # 이미지 영역을 추출합니다.
    images = page.get_image_info()
    image_rects = [sg.box(*image['bbox']) for image in images]

    # 도형과 이미지를 병합합니다.
    rect_list += image_rects

    merged_rects = _merge_rects(rect_list, distance=10, horizontal_distance=100)
    merged_rects = [rect for rect in merged_rects if explain_validity(rect) == 'Valid Geometry']

    # 큰 텍스트 영역과 작은 텍스트 영역을 분리하여 처리합니다: 큰 텍스트는 작은 텍스트와 병합하고, 작은 텍스트는 가까운 곳에 병합합니다.
    is_large_content = lambda x: (len(x[4]) / max(1, len(x[4].split('\n')))) > 5
    small_text_area_rects = [sg.box(*x[:4]) for x in page.get_text('blocks') if not is_large_content(x)]
    large_text_area_rects = [sg.box(*x[:4]) for x in page.get_text('blocks') if is_large_content(x)]
    _, merged_rects = _adsorb_rects_to_rects(large_text_area_rects, merged_rects, distance=0.1) # 완전히 겹침
    _, merged_rects = _adsorb_rects_to_rects(small_text_area_rects, merged_rects, distance=5) # 가까움

    # 다시 병합
    merged_rects = _merge_rects(merged_rects, distance=10)

    # 비교적 작은 사각형을 필터링합니다.
    merged_rects = [rect for rect in merged_rects if rect.bounds[2] - rect.bounds[0] > 20 and rect.bounds[3] - rect.bounds[1] > 20]

    return [rect.bounds for rect in merged_rects]


def _parse_pdf_to_images(pdf_path: str, output_dir: str = './') -> List[Tuple[str, List[str]]]:
    """
    PDF를 이미지로 파싱하고 output_dir에 저장합니다.
    """
    # PDF 파일을 엽니다.
    pdf_document = fitz.open(pdf_path)
    image_infos = []

    for page_index, page in enumerate(pdf_document):
        logging.info(f'페이지 파싱 중: {page_index}')
        rect_images = []
        rects = _parse_rects(page)
        for index, rect in enumerate(rects):
            fitz_rect = fitz.Rect(rect)

            # 페이지를 이미지로 저장
            pix = page.get_pixmap(clip=fitz_rect, matrix=fitz.Matrix(4, 4))
            name = f'{page_index}_{index}.png'
            pix.save(os.path.join(output_dir, name))
            rect_images.append(name)

            # 페이지에 빨간색 사각형을 그림 
            big_fitz_rect = fitz.Rect(fitz_rect.x0 - 1, fitz_rect.y0 - 1, fitz_rect.x1 + 1, fitz_rect.y1 + 1)

            # 빈 사각형
            page.draw_rect(big_fitz_rect, color=(1, 0, 0), width=1)

            # 사각형 영역(채움)
            # page.draw_rect(big_fitz_rect, color=(1, 0, 0), fill=(1, 0, 0))
            # 사각형 내 왼쪽 상단에 사각형의 인덱스 이름을 약간의 오프셋을 추가하여 작성합니다.
            text_x = fitz_rect.x0 + 2
            text_y = fitz_rect.y0 + 10
            text_rect = fitz.Rect(text_x, text_y - 9, text_x + 80, text_y + 2)

            # 흰색 배경 사각형을 그립니다.
            page.draw_rect(text_rect, color=(1, 1, 1), fill=(1, 1, 1))

            # 흰색 배경의 텍스트를 삽입합니다.
            page.insert_text((text_x, text_y), name, fontsize=10, color=(1, 0, 0))
            
        page_image_with_rects = page.get_pixmap(matrix=fitz.Matrix(3, 3))
        page_image = os.path.join(output_dir, f'{page_index}.png')
        page_image_with_rects.save(page_image)
        image_infos.append((page_image, rect_images))

    pdf_document.close()
    return image_infos


def _gpt_parse_images(
        image_infos: List[Tuple[str, List[str]]],
        prompt_dict: Optional[Dict] = None,
        output_dir: str = './',
        api_key: Optional[str] = None,
        base_url: Optional[str] = None,
        model: str = 'gpt-4o',
        verbose: bool = False,
        gpt_worker: int = 1,
        **args
) -> str:
    """
    이미지를 마크다운 내용으로 파싱합니다.
    """
    from GeneralAgent import Agent

    if isinstance(prompt_dict, dict) and 'prompt' in prompt_dict:
        prompt = prompt_dict['prompt']
        logging.info("프롬프트가 제공되었습니다. 사용자 프롬프트를 사용합니다.")
    else:
        prompt = DEFAULT_PROMPT
        logging.info("프롬프트가 제공되지 않았습니다. 기본 프롬프트를 사용합니다.")
    if isinstance(prompt_dict, dict) and 'rect_prompt' in prompt_dict:
        rect_prompt = prompt_dict['rect_prompt']
        logging.info("rect_prompt가 제공되었습니다. 사용자 프롬프트를 사용합니다.")
    else:
        rect_prompt = DEFAULT_RECT_PROMPT
        logging.info("rect_prompt가 제공되지 않았습니다. 기본 프롬프트를 사용합니다.")
    if isinstance(prompt_dict, dict) and 'role_prompt' in prompt_dict:
        role_prompt = prompt_dict['role_prompt']
        logging.info("role_prompt가 제공되었습니다. 사용자 프롬프트를 사용합니다.")
    else:
        role_prompt = DEFAULT_ROLE_PROMPT
        logging.info("role_prompt가 제공되지 않았습니다. 기본 프롬프트를 사용합니다.")

    def _process_page(index: int, image_info: Tuple[str, List[str]]) -> Tuple[int, str]:
        logging.info(f'gpt 페이지 파싱 중: {index}')
        agent = Agent(role=role_prompt, api_key=api_key, base_url=base_url, disable_python_run=True, model=model, **args)
        page_image, rect_images = image_info
        local_prompt = prompt
        if rect_images:
            local_prompt += rect_prompt + ', '.join(rect_images)
        content = agent.run([local_prompt, {'image': page_image}], display=verbose)
        return index, content

    contents = [None] * len(image_infos)
    with concurrent.futures.ThreadPoolExecutor(max_workers=gpt_worker) as executor:
        futures = [executor.submit(_process_page, index, image_info) for index, image_info in enumerate(image_infos)]
        for future in concurrent.futures.as_completed(futures):
            index, content = future.result()

            # 일부 경우 대형 모델이 여전히 ```markdown ``` 문자열을 출력합니다.
            if '```markdown' in content:
                content = content.replace('```markdown\n', '')
                last_backticks_pos = content.rfind('```')
                if last_backticks_pos != -1:
                    content = content[:last_backticks_pos] + content[last_backticks_pos + 3:]

            contents[index] = content

    output_path = os.path.join(output_dir, 'output.md')
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write('\n\n'.join(contents))

    return '\n\n'.join(contents)


def parse_pdf(
        pdf_path: str,
        output_dir: str = './',
        prompt: Optional[Dict] = None,
        api_key: Optional[str] = None,
        base_url: Optional[str] = None,
        model: str = 'gpt-4o',
        verbose: bool = False,
        gpt_worker: int = 1,
        **args
) -> Tuple[str, List[str]]:
    """
    PDF 파일을 마크다운 파일로 파싱합니다.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    image_infos = _parse_pdf_to_images(pdf_path, output_dir=output_dir)
    content = _gpt_parse_images(
        image_infos=image_infos,
        output_dir=output_dir,
        prompt_dict=prompt,
        api_key=api_key,
        base_url=base_url,
        model=model,
        verbose=verbose,
        gpt_worker=gpt_worker,
        **args
    )

    all_rect_images = []
    # 모든 사각형 이미지를 제거합니다.
    if not verbose:
        for page_image, rect_images in image_infos:
            if os.path.exists(page_image):
                os.remove(page_image)
            all_rect_images.extend(rect_images)
    return content, all_rect_images